# WeatherPy
----

### Analysis
* Cities closer to the equador have higher temperatures but, interestingly, latitudes further from the equador in the north have lower temperatures than those further from the equador in the south. This is most likely because there are simply more cities in the northern end of the hemisphere, while the southern hemisphere is Antarctica, where there are no cities.
* There are no clear patterns or relationships between latitude and humidity, cloudiness, or wind speed. Nor are there any clear relationships between longitude and any of the four measures of weather.  
* Interestingly, cloudiness seems to cluster around 0% and 100%.


In [4]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import datetime as date

# Import API key
from config import key

# Incorporated citipy to determine city based on latitude and longitude

from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

In [6]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)


643

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it's being processed (with the city number and city name).


In [7]:
url = "http://api.openweathermap.org/data/2.5/weather?"
units = "imperial"

query_url = f"{url}appid={key}&units={units}&q="
response = requests.get(query_url+cities[0]).json()

lats = []
temp = []
hums = []
cloud =[]
wndspd= []
country =[]
date = []
longs = []
good_cities = []

recordcount = 1
timeoutcounter = 1
print("Beginning Data Retrieval")     
print("-----------------------------")
for city in cities:
    response = requests.get(query_url + city).json()
    print(f"Processing Record {recordcount}: {city}")
    if timeoutcounter%10 == 0:
        time.sleep(5)
        
    try:
        lats.append(response["coord"]["lat"])
        temp.append(response['main']['temp'])
        hums.append(response['main']['humidity'])
        cloud.append(response['clouds']["all"])
        wndspd.append(response['wind']["speed"])
        country.append(response["sys"]["country"])
        date.append(response["dt"])
        longs.append(response["coord"]["lon"])
        good_cities.append(city)
        timeoutcounter += 1
    
    except (KeyError, IndexError):
        print("City not found. Skipping...")
        cities.remove(city)
        pass
    recordcount = recordcount+1

Beginning Data Retrieval
-----------------------------
Processing Record 1: new norfolk
Processing Record 2: vaini
Processing Record 3: punta arenas
Processing Record 4: marsh harbour
Processing Record 5: mullaitivu
City not found. Skipping...
Processing Record 6: laguna
Processing Record 7: taolanaro
City not found. Skipping...
Processing Record 8: kodiak
Processing Record 9: okhotsk
Processing Record 10: rikitea
Processing Record 11: brae
Processing Record 12: tual
Processing Record 13: havre-saint-pierre
Processing Record 14: tiksi
Processing Record 15: yellowknife
Processing Record 16: hellvik
Processing Record 17: demmin
Processing Record 18: paamiut
Processing Record 19: faanui
Processing Record 20: chokurdakh
Processing Record 21: coquimbo
Processing Record 22: seminole
Processing Record 23: leshukonskoye
Processing Record 24: nouadhibou
Processing Record 25: sao raimundo das mangabeiras
Processing Record 26: ribeira grande
Processing Record 27: saskylakh
Processing Record 28: s

Processing Record 237: harper
Processing Record 238: masunga
Processing Record 239: airai
Processing Record 240: barstow
Processing Record 241: tessalit
Processing Record 242: talas
Processing Record 243: sao joao da barra
Processing Record 244: bambous virieux
Processing Record 245: umzimvubu
City not found. Skipping...
Processing Record 246: mecca
Processing Record 247: caravelas
Processing Record 248: vygonichi
Processing Record 249: suifenhe
Processing Record 250: andenes
City not found. Skipping...
Processing Record 251: tigil
Processing Record 252: puerto penasco
Processing Record 253: aykhal
Processing Record 254: tungkang
City not found. Skipping...
Processing Record 255: asau
City not found. Skipping...
Processing Record 256: san cristobal
Processing Record 257: port hawkesbury
Processing Record 258: nalut
Processing Record 259: samandag
City not found. Skipping...
Processing Record 260: vaitupu
City not found. Skipping...
Processing Record 261: dalvik
Processing Record 262: o

Processing Record 464: loei
Processing Record 465: ghanzi
Processing Record 466: linchuan
City not found. Skipping...
Processing Record 467: jaru
Processing Record 468: lavrentiya
Processing Record 469: nikel
Processing Record 470: khandyga
Processing Record 471: gardan diwal
City not found. Skipping...
Processing Record 472: prabumulih
Processing Record 473: chokwe
City not found. Skipping...
Processing Record 474: stony mountain
Processing Record 475: conceicao do araguaia
Processing Record 476: salaga
Processing Record 477: ust-nera
Processing Record 478: kununurra
Processing Record 479: kuryk
Processing Record 480: scottsbluff
Processing Record 481: ylojarvi
Processing Record 482: mao
Processing Record 483: oak harbor
Processing Record 484: imbituba
Processing Record 485: gao
Processing Record 486: kilindoni
Processing Record 487: verkhnyaya toyma
Processing Record 488: pangnirtung
Processing Record 489: homer
Processing Record 490: guna
Processing Record 491: metlaltoyuca
Processi

### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [8]:
weather_dict = {
    "City": good_cities,
    "Latitude": lats,
    "Longitude": longs,
    "Temperature": temp,
    "Humidity":hums,
    "Cloudiness":cloud,
    "Wind Speed":wndspd,
    "Country":country,
    "Date":date
}

weather_data = pd.DataFrame(weather_dict)
weather_data.to_csv(r'../Assets/weather_data.csv')
weather_data.head()

,City,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
0,new norfolk,-42.78,147.06,65.77,77,75,3.36,AU,1569036937
1,vaini,15.34,74.49,73.40,88,40,11.41,IN,1569036940
2,punta arenas,-53.16,-70.91,33.37,74,20,24.16,CL,1569036719
3,marsh harbour,26.54,-77.06,79.90,79,100,28.32,BS,1569036942
4,laguna,16.17,-88.94,77.00,100,0,2.80,BZ,1569036944
